In [15]:
import pandas as pd
from sqlalchemy import create_engine
import datetime


In [16]:
fire_file = "California_Wildland_Fire_Perimeters.csv"
fire_df = pd.read_csv(fire_file)
fire_df.head()

,OBJECTID,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,Shape__Area,Shape__Length
0,21440,2020.0,CA,CDF,NEU,NELSON,00013212,2020/06/18 00:00:00+00,2020/06/23 00:00:00+00,11.0,NaN,110.0,109.602280,1.0,1.0,NaN,7.331347e+05,4179.743142
1,21441,2020.0,CA,CDF,NEU,AMORUSO,00011799,2020/06/01 00:00:00+00,2020/06/04 00:00:00+00,2.0,NaN,670.0,685.585022,1.0,1.0,NaN,4.578172e+06,12399.375391
2,21442,2020.0,CA,CDF,NEU,ATHENS,00018493,2020/08/10 00:00:00+00,2020/03/01 00:00:00+00,14.0,NaN,26.0,27.300480,1.0,1.0,NaN,1.823876e+05,2119.194120
3,21443,2020.0,CA,CDF,NEU,FLEMING,00007619,2020/03/31 00:00:00+00,2020/04/01 00:00:00+00,9.0,NaN,13.0,12.931545,1.0,1.0,NaN,8.667942e+04,2029.524881
4,21444,2020.0,CA,CDF,NEU,MELANESE,00008471,2020/04/14 00:00:00+00,2020/04/19 00:00:00+00,18.0,NaN,10.3,10.315964,1.0,1.0,NaN,7.017912e+04,1342.742903


In [17]:
datetime.datetime.strptime(fire_df[Year_], "%d/%m/%Y").strftime("%Y-%m-%d")

'2008-12-21'

In [18]:
drought_file = "USDM-california.csv"
drought_df = pd.read_csv(drought_file)
drought_df.head()

,MapDate,StateAbbreviation,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20220607,CA,0.0,100.0,99.79,97.48,59.81,11.59,2022-06-07,2022-06-13,1
1,20220531,CA,0.0,100.0,99.86,97.56,59.81,11.59,2022-05-31,2022-06-06,1
2,20220524,CA,0.0,100.0,99.86,97.56,59.81,11.59,2022-05-24,2022-05-30,1
3,20220517,CA,0.0,100.0,99.86,95.14,59.81,0.18,2022-05-17,2022-05-23,1
4,20220510,CA,0.0,100.0,99.86,95.14,59.81,0.00,2022-05-10,2022-05-16,1


# Rename the column headers
fire_transformed = fire_transformed.rename(columns={"Object Id": "OBJECTID", 
                                                            "Year": "YEAR_", 
                                                            "State": "STATE", 
                                                            "Agency": "AGENCY", 
                                                            "Unit ID": "UNIT_ID", 
                                                            "Alarm Date": "ALARM_DATE", 
                                                            "Containment Date": "CONT_DATE", 
                                                            "Cause": "CAUSE", 
                                                            "Comments": "COMMENTS", 
                                                            "Reported Acreage": "REPORT_AC", 
                                                            "GIS Acres": "GIS_ACRES", 
                                                            "Collection Method": "C_METHOD", 
                                                            "Objective": "OBJECTIVE", 
                                                            "Shape Area": "Shape__Area", 
                                                            "Shape Length": "Shape__Length"})

In [22]:
# Create a filtered dataframe from specific columns
fire_cols = ["OBJECTID", "YEAR_", "STATE", "AGENCY", "UNIT_ID", "ALARM_DATE", "CONT_DATE", "CAUSE", "COMMENTS", "REPORT_AC", "GIS_ACRES", "C_METHOD", "OBJECTIVE", "Shape__Area", "Shape__Length"]

fire_transformed = fire_df[fire_cols].copy()

# Clean the data by dropping duplicates and setting the index

fire_transformed.set_index("Object ID", inplace=True)
fire_transformed.head()

KeyError: "None of ['Object ID'] are in the columns"